In [2]:
import numpy as np
import cv2
import sys
import time

In [103]:
# You can use this function by providing the marker corners, the pixel coordinates of the point of interest
#(x is increasing to the right and y is increasing downward)
# the length of the ArUco marker in m, the camera matrix, and the distortion coefficients. It will return the 
# distance from the ArUco marker to the point of interest in m.

def distance_to_aruco(marker_corners, point_of_interest, marker_length, camera_matrix, dist_coeffs, marker_id):
    # Obtain ArUco pose estimation
    rvecs, tvecs, ids = cv2.aruco.estimatePoseSingleMarkers(marker_corners, marker_length, camera_matrix, dist_coeffs)
    
    # Check if any markers are detected
    if ids is None:
        return None
    
    # Find the index of the marker with the specified ID
    marker_index = np.where(ids == marker_id)[0]
    
    # Check if the specified marker ID is detected
    if len(marker_index) == 0:
        return None
    
    # Extract the pose of the marker with the specified ID
    marker_index = marker_index[0]  # Take the first marker with the specified ID
    rvec_marker, tvec_marker = rvecs[marker_index], tvecs[marker_index]
    
    # Convert pixel coordinates of the point of interest to camera coordinates
    homogeneous_pixel_coords = np.array([[point_of_interest[0]], [point_of_interest[1]], [1]])
    homogeneous_camera_coords = np.dot(np.linalg.inv(camera_matrix), homogeneous_pixel_coords)
    camera_coords = homogeneous_camera_coords / homogeneous_camera_coords[2]
    
    # Transform camera coordinates to marker coordinates for the marker with the specified ID
    rotation_matrix_marker, _ = cv2.Rodrigues(rvec_marker)
    marker_coords = np.dot(rotation_matrix_marker.T, (camera_coords - tvec_marker.T))
    
    # Return the vector instead of the absolute distance
    return marker_coords


In [104]:
# This function will take an image and overlay evenly spaced circles within a specified box area.
# Also it appends 
#Note: 
    # origin(0,0) of picture is at top-left corner
    # x-axis increases to the right
    # y-axis increases downward
    
def overlay_circles_on_image(image, num_points, circle_radius, circle_color, top_left, top_right, bottom_left, bottom_right):
    # Convert numpy array to PIL Image
    image_pil = Image.fromarray(image)
    draw = ImageDraw.Draw(image_pil)
    
    # Initialize an empty list to store the coordinates of the points
    point_coordinates = []
    
    height = bottom_left[1] - top_left[1]
    width = top_right[0] - top_left[0]
    
    y_points = np.linspace(top_left[1], bottom_left[1], num_points).astype(int)
    x_points_left_to_right = np.linspace(top_left[0], top_right[0], num_points).astype(int)
    x_points_right_to_left = np.linspace(top_right[0], top_left[0], num_points).astype(int)
    
    for y in y_points:
        # Iterate from left to right
        for x in x_points_left_to_right:
            left_up_point = (x - circle_radius, y - circle_radius)
            right_down_point = (x + circle_radius, y + circle_radius)
            
            # Append the coordinates of the current point to the list
            point_coordinates.append((x, y))
            
            draw.ellipse([left_up_point, right_down_point], fill=circle_color)
        
        # Switch to iterating from right to left for the next row
        x_points_left_to_right, x_points_right_to_left = x_points_right_to_left, x_points_left_to_right[::-1]
    
    # Convert the modified PIL Image back to a numpy array
    modified_image = np.array(image_pil)
    
    # Return both the modified image and the list of point coordinates
    return modified_image, point_coordinates


In [105]:
# calculates the relative rotation angles between two markers represented by their rotation 
# vectors rvec1 and rvec2. ( vec1 and rvec2 come from pose_estimation function.

def angle_between_markers(rvec1, rvec2):
    # Calculate relative rotation vectors
    relative_rvec1 = rvec1 - np.array([0, 0, 0])  # Assume origin is at (0, 0, 0)
    relative_rvec2 = rvec2 - np.array([0, 0, 0])  # Assume origin is at (0, 0, 0)

    # Convert relative rotation vectors to rotation matrices
    rotation_matrix1, _ = cv2.Rodrigues(relative_rvec1)
    rotation_matrix2, _ = cv2.Rodrigues(relative_rvec2)

    # Compute the rotation difference between the two markers
    relative_rotation_matrix = np.dot(rotation_matrix2, np.linalg.inv(rotation_matrix1))
        
    # Convert rotation matrix to Euler angles
    relative_euler_angles, _ = cv2.Rodrigues(relative_rotation_matrix)
     
    # Convert to Angles in x, y, and z
    theta_x = np.arctan2(relative_rotation_matrix[2, 1], relative_rotation_matrix[2, 2])
    theta_y = np.arctan2(-relative_rotation_matrix[2, 0], np.sqrt(relative_rotation_matrix[2, 1]**2 + relative_rotation_matrix[2, 2]**2))
    theta_z = np.arctan2(relative_rotation_matrix[1, 0], relative_rotation_matrix[0, 0])
    
    # Convert to degrees
    theta_x_deg = np.degrees(theta_x)
    theta_y_deg = np.degrees(theta_y)
    theta_z_deg = np.degrees(theta_z)
    
    # Make it a single matrix
    Theta = np.array([theta_x_deg, theta_y_deg, theta_z_deg])
        
    return Theta

In [106]:
# Finds the absolute distance between two aruco markers
def distance_between_markers(tvec1, tvec2):
    distance = np.linalg.norm(tvec1 - tvec2)
    return distance


In [107]:
def draw_axis(img, rvec, tvec, camera_matrix, dist_coeffs, length):
    axis_points = np.float32([[0,0,0], [length,0,0], [0,length,0], [0,0,length]]).reshape(-1,3)

    # Project axis points to the image plane
    img_points, _ = cv2.projectPoints(axis_points, rvec, tvec, camera_matrix, dist_coeffs)

    # Convert image points to integers
    img_points = np.round(img_points).astype(int)

    # Draw lines
    img = cv2.line(img, tuple(img_points[0].ravel()), tuple(img_points[1].ravel()), (0,0,255), 2)  # x-axis (red)
    img = cv2.line(img, tuple(img_points[0].ravel()), tuple(img_points[2].ravel()), (0,255,0), 2)  # y-axis (green)
    img = cv2.line(img, tuple(img_points[0].ravel()), tuple(img_points[3].ravel()), (255,0,0), 2)  # z-axis (blue)

    return img

In [108]:
def aruco_display(corners, ids, rejected, image):  
	if len(corners) > 0:
		
		ids = ids.flatten()
		
		for (markerCorner, markerID) in zip(corners, ids):
			
			corners = markerCorner.reshape((4, 2))
			(topLeft, topRight, bottomRight, bottomLeft) = corners
			
			topRight = (int(topRight[0]), int(topRight[1]))
			bottomRight = (int(bottomRight[0]), int(bottomRight[1]))
			bottomLeft = (int(bottomLeft[0]), int(bottomLeft[1]))
			topLeft = (int(topLeft[0]), int(topLeft[1]))

			cv2.line(image, topLeft, topRight, (0, 255, 0), 2)
			cv2.line(image, topRight, bottomRight, (0, 255, 0), 2)
			cv2.line(image, bottomRight, bottomLeft, (0, 255, 0), 2)
			cv2.line(image, bottomLeft, topLeft, (0, 255, 0), 2)
			
			cX = int((topLeft[0] + bottomRight[0]) / 2.0)
			cY = int((topLeft[1] + bottomRight[1]) / 2.0)
			cv2.circle(image, (cX, cY), 4, (0, 0, 255), -1)
			
			cv2.putText(image, str(markerID),(topLeft[0], topLeft[1] - 10), cv2.FONT_HERSHEY_SIMPLEX,
				0.5, (0, 255, 0), 2)
			print("[Inference] ArUco marker ID: {}".format(markerID))
			
	return image

In [114]:
def pose_estimation(frame, aruco_dict_type, matrix_coefficients, distortion_coefficients):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    aruco_dict = cv2.aruco.getPredefinedDictionary(aruco_dict_type)  # Define aruco_dict here
    parameters = cv2.aruco.DetectorParameters()

    corners, ids, rejected_img_points = cv2.aruco.detectMarkers(gray, aruco_dict, parameters=parameters)

    # Call aruco_display to draw markers and their IDs
    frame = aruco_display(corners, ids, rejected_img_points, frame)

    if len(corners) > 1:
        print("There are two or more markers")
        # Estimate pose for the first marker
        rvec1, tvec1, _ = cv2.aruco.estimatePoseSingleMarkers(corners[0], 0.175, matrix_coefficients, distortion_coefficients)

        # Estimate pose for the second marker
        rvec2, tvec2, _ = cv2.aruco.estimatePoseSingleMarkers(corners[1], 0.175, matrix_coefficients, distortion_coefficients)
        
        # Calculating Relative Position Vectors
        distance_vector = tvec1-tvec2;
        distance = distance_between_markers(tvec1.squeeze(), tvec2.squeeze())
        print("Distance between markers:", distance)
        print("Distance vector between markers:", distance_vector)
        
        # Calculating Relative Rotations
        Theta = angle_between_markers(rvec1.squeeze(), rvec2.squeeze())  
        #print(f"Relative Rotation Angle around X-axis (degrees): {Theta[0]}")
        #print(f"Relative Rotation Angle around Y-axis (degrees): {Theta[1]}")
        print(f"Relative Rotation Angle around Z-axis (degrees): {Theta[2]}")
        
        dist1stpt = distance_to_aruco(corners, (500,100), 0.175, intrinsic_camera, distortion,153)
        print("Distance between Aruco marker and point:", dist1stpt)
        
        cv2.aruco.drawDetectedMarkers(frame, corners)
        draw_axis(frame, rvec1, tvec1, matrix_coefficients, distortion_coefficients, 0.1)
        draw_axis(frame, rvec2, tvec2, matrix_coefficients, distortion_coefficients, 0.1)

    return frame, distance_vector



In [115]:
import numpy as np
import cv2
from PIL import Image, ImageDraw

ARUCO_DICT = {"DICT_ARUCO_ORIGINAL": cv2.aruco.DICT_ARUCO_ORIGINAL,}

aruco_type = "DICT_ARUCO_ORIGINAL"

arucoDict = cv2.aruco.getPredefinedDictionary(ARUCO_DICT[aruco_type])

arucoParams = cv2.aruco.DetectorParameters()


intrinsic_camera = np.array(((933.15867, 0, 657.59),(0,933.1586, 400.36993),(0,0,1)))
distortion = np.array((-0.43948,0.18514,0,0))


cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

# While loop for computer vision, currently calculates distance between markers only
while cap.isOpened():
     
    ret, img = cap.read()
    
    # Calculates Position of Arucos and outputs the image with the axises and ids overlaid
    output = pose_estimation(img, ARUCO_DICT[aruco_type], intrinsic_camera, distortion)
    
    # Add the Circle Overlay
    processed_image, point_coordinates = overlay_circles_on_image(
        output,
        num_points=10,
        circle_radius=3,
        circle_color=(255, 215, 0, 255),
        top_left=(500, 100),
        top_right=(1000, 100),
        bottom_left=(500, 500),
        bottom_right=(1000, 500)
    )

#     # Calculate distance between the Aruco marker and the first point
#     dist1stpt = distance_to_aruco(corners, point_coordinates[0], 0.175, intrinsic_camera, distortion,153)
#     print("Distance between Aruco marker and point:", dist1stpt)
    cv2.imshow('Estimated Pose',  processed_image)

    # Get the corners and ids of detected markers
    corners, ids, _ = cv2.aruco.detectMarkers(img, arucoDict, parameters=arucoParams)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 0
There are two or more markers
Distance between markers: 0.7741959877048317
Distance vector between markers: [[[ 0.51076427 -0.1639386  -0.55823241]]]
Relative Rotation Angle around Z-axis (degrees): -2.049456785256332
Distance between Aruco marker and point: None
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 0
There are two or more markers
Distance between markers: 0.8131382561642967
Distance vector between markers: [[[ 0.51006556 -0.162544   -0.61205097]]]
Relative Rotation Angle around Z-axis (degrees): -2.0326092009710632
Distance between Aruco marker and point: None
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 0
There are two or more markers
Distance between markers: 0.783205187811614
Distance vector

[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 0
There are two or more markers
Distance between markers: 0.8227297563956532
Distance vector between markers: [[[ 0.50772268 -0.16911114 -0.62490267]]]
Relative Rotation Angle around Z-axis (degrees): 171.22809736800733
Distance between Aruco marker and point: None
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 0
There are two or more markers
Distance between markers: 0.8094530628395488
Distance vector between markers: [[[ 0.51049223 -0.16739404 -0.60546774]]]
Relative Rotation Angle around Z-axis (degrees): -1.3319094436373748
Distance between Aruco marker and point: None
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 0
There are two or more markers
Distance between markers: 0.8092961090045961
Distance vector between markers: [[[ 0.

[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 0
There are two or more markers
Distance between markers: 0.8092961090045961
Distance vector between markers: [[[ 0.51053906 -0.16793044 -0.60506977]]]
Relative Rotation Angle around Z-axis (degrees): -1.332622294642162
Distance between Aruco marker and point: None
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 0
There are two or more markers
Distance between markers: 0.8139385400650191
Distance vector between markers: [[[ 0.50767338 -0.16847588 -0.61349781]]]
Relative Rotation Angle around Z-axis (degrees): -1.0068915918655192
Distance between Aruco marker and point: None
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 0
There are two or more markers
Distance between markers: 0.8092961090045961
Distance vector between markers: [[[ 0.51053906 -0.16793044 -0.60506977]]]
Relative Rotation Angle around Z-axis (degrees): -1.332622294642162
Distance between Aruco marker 

Distance between markers: 0.8405757243360515
Distance vector between markers: [[[ 0.49901805 -0.16973258 -0.65478194]]]
Relative Rotation Angle around Z-axis (degrees): -0.4586834028853009
Distance between Aruco marker and point: None
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 0
There are two or more markers
Distance between markers: 0.8405757243360515
Distance vector between markers: [[[ 0.49901805 -0.16973258 -0.65478194]]]
Relative Rotation Angle around Z-axis (degrees): -0.4586834028853009
Distance between Aruco marker and point: None
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 0
There are two or more markers
Distance between markers: 0.8158824405224525
Distance vector between markers: [[[ 0.50741342 -0.16852111 -0.61627625]]]
Relative Rotation Angle around Z-axis (degrees): -1.7200004944043001
Distance between Aruco marker and point: None
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 0
[Inference

[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 0
There are two or more markers
Distance between markers: 0.7978594853307475
Distance vector between markers: [[[ 0.51007125 -0.16622996 -0.59057148]]]
Relative Rotation Angle around Z-axis (degrees): -2.3543894551997093
Distance between Aruco marker and point: None
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 0
There are two or more markers
Distance between markers: 0.8236822497709178
Distance vector between markers: [[[ 0.50148028 -0.16614977 -0.63195271]]]
Relative Rotation Angle around Z-axis (degrees): -1.079287745881358
Distance between Aruco marker and point: None
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 0
There are two or more markers
Distance between markers: 0.8094530628395488
Distance vector between markers: [[[ 0.51049223 -0.16739404 -0.60546774]]]
Relative Rotation Angle around Z-axis (degrees): -1.33190944363

...

[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
There are two or more markers
Distance between markers: 0.8780425824010589
Distance vector between markers: [[[-0.82570892  0.03326014 -0.29674453]]]
Relative Rotation Angle around Z-axis (degrees): 0.07479689708789854
Distance between Aruco marker and point: None
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
There are two or more markers
Distance between markers: 0.8699131570594361
Distance vector between markers: [[[-0.82322231  0.0332579  -0.27919142]]]
Relative Rotation Angle around Z-axis (degrees): -0.20022563158267911
Distance between Aruco marker and point: None
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
There are two or more markers
Distance between markers: 0.882560045759768
Distance vector between markers: [[[-0.82589192  0.03265184 -0.30943274]]]
Relative Rotation Angle around Z-axis (degrees): -0.712964189681671
Distance between Aruco marker and point: None
[Inference] Ar

[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
There are two or more markers
Distance between markers: 0.8698954560856722
Distance vector between markers: [[[-0.82318248  0.03299772 -0.27928455]]]
Relative Rotation Angle around Z-axis (degrees): -0.18855124175132743
Distance between Aruco marker and point: None
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
There are two or more markers
Distance between markers: 0.8698954560856722
Distance vector between markers: [[[-0.82318248  0.03299772 -0.27928455]]]
Relative Rotation Angle around Z-axis (degrees): -0.18855124175132743
Distance between Aruco marker and point: None
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
There are two or more markers
Distance between markers: 0.8698954560856722
Distance vector between markers: [[[-0.82318248  0.03299772 -0.27928455]]]
Relative Rotation Angle around Z-axis (degrees): -0.18855124175132743
Distance between Aruco marker and point: None
[Inference

[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
There are two or more markers
Distance between markers: 0.8699580109787529
Distance vector between markers: [[[-0.82074882  0.03510871 -0.28629651]]]
Relative Rotation Angle around Z-axis (degrees): -7.569291969415975
Distance between Aruco marker and point: None
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
There are two or more markers
Distance between markers: 0.8709468330249502
Distance vector between markers: [[[-0.82009961  0.03565606 -0.29105611]]]
Relative Rotation Angle around Z-axis (degrees): -8.641050486261149
Distance between Aruco marker and point: None
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
There are two or more markers
Distance between markers: 0.8699303297778092
Distance vector between markers: [[[-0.82326179  0.03351793 -0.27909738]]]
Relative Rotation Angle around Z-axis (degrees): -0.2118064758831374
Distance between Aruco marke

[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
There are two or more markers
Distance between markers: 0.8719160075074263
Distance vector between markers: [[[-0.82145086  0.034246   -0.29031574]]]
Relative Rotation Angle around Z-axis (degrees): -0.23626857273634902
Distance between Aruco marker and point: None
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
There are two or more markers
Distance between markers: 0.8692725320564068
Distance vector between markers: [[[-0.82372793  0.03363529 -0.27563689]]]
Relative Rotation Angle around Z-axis (degrees): -1.0473107275297202
Distance between Aruco marker and point: None
[Inference] ArUco marker ID: 0
[Inference] ArUco marker ID: 153
There are two or more markers
Distance between markers: 0.8665952521036736
Distance vector between markers: [[[-0.8236261   0.03356974 -0.26739569]]]
Relative Rotation Angle around Z-axis (degrees): -1.0787512647714919
Distance between Aruco marker and point: None
[Inference] 

[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 0
There are two or more markers
Distance between markers: 0.9816561126156286
Distance vector between markers: [[[ 0.66193961 -0.05770097 -0.72260312]]]
Relative Rotation Angle around Z-axis (degrees): -15.553954490977095
Distance between Aruco marker and point: None
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 0
There are two or more markers
Distance between markers: 0.9727840357598363
Distance vector between markers: [[[ 0.66381134 -0.0569183  -0.70881845]]]
Relative Rotation Angle around Z-axis (degrees): -7.260112964316653
Distance between Aruco marker and point: None
[Inference] ArUco marker ID: 153
[Inference] ArUco marker ID: 0
There are two or more markers
Distance between markers: 0.9762672149577638
Distance vector between markers: [[[ 0.65758575 -0.05454821 -0.71951591]]]
Relative Rotation Angle around Z-axis (degrees): 11.280167191813653
Distance between Aruco marker and point: None
[Inference] ArU

In [84]:
point_coordinates

[(500, 100),
 (555, 100),
 (611, 100),
 (666, 100),
 (722, 100),
 (777, 100),
 (833, 100),
 (888, 100),
 (944, 100),
 (1000, 100),
 (1000, 144),
 (944, 144),
 (888, 144),
 (833, 144),
 (777, 144),
 (722, 144),
 (666, 144),
 (611, 144),
 (555, 144),
 (500, 144),
 (1000, 188),
 (944, 188),
 (888, 188),
 (833, 188),
 (777, 188),
 (722, 188),
 (666, 188),
 (611, 188),
 (555, 188),
 (500, 188),
 (500, 233),
 (555, 233),
 (611, 233),
 (666, 233),
 (722, 233),
 (777, 233),
 (833, 233),
 (888, 233),
 (944, 233),
 (1000, 233),
 (500, 277),
 (555, 277),
 (611, 277),
 (666, 277),
 (722, 277),
 (777, 277),
 (833, 277),
 (888, 277),
 (944, 277),
 (1000, 277),
 (1000, 322),
 (944, 322),
 (888, 322),
 (833, 322),
 (777, 322),
 (722, 322),
 (666, 322),
 (611, 322),
 (555, 322),
 (500, 322),
 (1000, 366),
 (944, 366),
 (888, 366),
 (833, 366),
 (777, 366),
 (722, 366),
 (666, 366),
 (611, 366),
 (555, 366),
 (500, 366),
 (500, 411),
 (555, 411),
 (611, 411),
 (666, 411),
 (722, 411),
 (777, 411),
 (833